<a href="https://colab.research.google.com/github/karlssoj/cv26/blob/main/X_Y_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Klassificeringsexempel med faltningsnätverk (CNN) implementerat med Keras och Tensorflow </h1>

I det här simpla exemplet tränar vi in en modell att kunna se skillnad mellan X och O. I katalogen X_O_training finns 2 kataloger O och X. Vardera katalog innehåller 34 träningsbilder på X respektive O. Sedan har vi också en annan katalog X_O_test som innehåller 10 stycen X och 10 stycken 0. Dessa används för att validera modellens prestanda

Skriptet är testkört i google colab (men kan nog även köras lokalt).

**Börjar med att klona repon så att vi får med alla filer**

In [37]:
!git clone -b main https://github.com/karlssoj/cv26.git

Cloning into 'cv26'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 113 (delta 8), reused 109 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 582.04 KiB | 3.53 MiB/s, done.
Resolving deltas: 100% (8/8), done.


<b> Importerar OpenCV och Matplotlib



In [38]:
import matplotlib.pyplot as plt
import cv2 as cv

<b> Importerar ImageDataGenerator från Keras-biblioteket för att kunna hantera träningsbilderna

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

<b> Skapar ett ImageDataGenerator-objekt med vissa inställningar för data augumentering

In [40]:
image_gen = ImageDataGenerator(rotation_range=30,    #slumpmässiga roteringar av bilderna i grader
                             width_shift_range=0.1,  #slumpmässig skiftning i x-led (ett värde mellan 0 och 1)
                             height_shift_range=0.1, #slumpmässig skiftning i y-led (ett värde mellan 0 och 1)
                             rescale=1/255,          #skalar ner ett 8-bitars pixelvärde (0-255) till ett värde mellan 0 och 1
                             zoom_range=0.6,         #slumpmässig zoom
                             horizontal_flip=True)   #"flippar" bilden slumpmässigt i horisontalt läge

<B> Delar in träningsbilderna i validation 20% och training 80% samt läser även in testbilderna till objektet test_image_gen

In [ ]:
image_gen = ImageDataGenerator(rescale=1/255, validation_split=0.2)

train_image_gen = image_gen.flow_from_directory('/content/cv26/X_O_training',
                                                target_size=(100,100),
                                                subset='training')

val_image_gen = image_gen.flow_from_directory('/content/cv26/X_O_training',
                                              target_size=(100,100),
                                              subset='validation')  # 20% val

test_image_gen = image_gen.flow_from_directory('/content/cv26/X_O_test',
                                               target_size=(100,100))  # 20% test


<b> Träningsbilderna har automatiskt markerats enligt namnet på katalogerna de finns sparade i, dvs. O har labeln 0 och X har labeln 1

In [ ]:
print(train_image_gen.class_indices)

<b> Importerar behövliga bibliotek från Keras för att kunna skapa en CNN-modell

In [42]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input

<b> Skapar ett faltningslager (convolutional layer) bestående av 32 olika filters. Som aktiveringsfunktion används ReLU (för att bli av med alla negativa pixelvärden<br><br>

<b> Kör MaxPooling med 2x2 fönsterstorlek och hopp (stride) = 2<br><br>

<b> Formaterar om alla filtrerade bilder (i detta fall 32 stycken) till en enda array = flatten<br><br>

<b> Till slut mynnar allt ut i 2 neuroner i "fully connected layer", en för X och en för Y

In [57]:
model = Sequential()
model.add(Input(shape=(100, 100, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))                            #Faltningslager med 32 filter
model.add(MaxPooling2D(pool_size = (2, 2)))                                 #MaxPooling 2x2 fönsterstorlek och hopp (stride) = 2
model.add(Flatten())                                                        #Formaterar om alla filtrerade blder (32 stycken) till en array
model.add(Dense(units = 2, activation = "sigmoid"))                         #2 neuroner på slutet

<b> "Kompilerar" modellen och definierar loss function och optimizer

In [58]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.summary()

<b> Startar själva inträning. Epochs definierar hur många ggr inlärningsalgoritmen kommer att gå igenom hela datasetet. Vi definierar även var valideringsbilderna finns som ska tas i beaktand under inträning

In [ ]:
results = model.fit(train_image_gen,
                    validation_data=val_image_gen,
                    epochs=100)

In [ ]:
plt.plot(results.history['accuracy'])

In [ ]:
plt.plot(results.history['val_accuracy'])

<b> Vi laddar in en testbild (En bild som inte finns i träningsdataset:en) för att se hur bra vår modell fungerar!

In [62]:
import numpy as np
import keras.utils as image
test_image = '/content/cv26/test_image2.jpg'
test_image = image.load_img(test_image, target_size=(100,100)) #laddar testbilden och sätter storleken till samma som träningsbilderna
test_image = image.img_to_array(test_image)                    #konverterar till array-format

test_image = np.expand_dims(test_image, axis=0)
test_image = test_image/255                                    #Ändrar pixelvärden från 8-bitar (0-255) till värden mellan 0 och 1

<b> Vi startar "predikteringen" dvs. Som svar får vi konfidensvärden som berättar hur stor sannolikhten är att bilden innehåller ett X och hur stor sannolikheten är att bilden innehåller ett O

In [ ]:
prob = model.predict(test_image)

In [ ]:
print(prob)

In [65]:
print("Sannolikheten att testbilden innehåller O: " + str(prob[0][0]))
print("Sannolikheten att testbilden innehåller X: " + str(prob[0][1]))

Sannolikheten att testbilden innehåller O: 0.03387692
Sannolikheten att testbilden innehåller X: 0.93606055


<B> Evaluerar vår klassificeringsmodell mot testdatasetet

In [ ]:
test_loss, test_acc = model.evaluate(test_image_gen)
print(f"Test Accuracy: {test_acc:.4f}")